### CIFAR10 Classifer Evaluation
In this notebook, we evaluate pre-trained **CNN** to classify images from the CIFAR-10 database.

The images in this database are small color images that fall into one of ten classes; some example images are pictured below.

<img src='notebook_ims/cifar_data.png' width=50% height=50% />

In [ ]:
!python ../tools/download_data_set.py --dataset_name cifar10

In [1]:
// Import necessary library headers.
#include <mlpack/xeus-cling.hpp>
#include <mlpack/core.hpp>
#include <mlpack/core/data/split_data.hpp>
#include <mlpack/methods/ann/layer/layer.hpp>
#include <mlpack/methods/ann/ffn.hpp>
#include <ensmallen.hpp>

In [2]:
using namespace mlpack;

In [3]:
using namespace mlpack::ann;

In [4]:
using namespace arma;

In [5]:
using namespace ens;

In [6]:
arma::Row<size_t> getLabels(const arma::mat& yPreds) 
{
    arma::Row<size_t> yLabels(yPreds.n_cols);
    for (arma::uword i = 0; i < yPreds.n_cols; ++i)
    {
        yLabels(i) = yPreds.col(i).index_max();
    }
    return yLabels;
}

### Network Architecture

Our Model using Convolutional Neural Network CNN architecture.
* Convolutional layers - which can be thought of as stack of filtered images.
* Maxpooling layers - which reduce the x-y size of an input, keeping only the most _active_ pixels from the previous layer.
* Linear layer - Applies a linear transformation to the incoming data: $ y=xA^T+b $

```
32 x 32 x 3 --- conv (6 feature maps of kernel size 5 x 5 with stride = 1) ---> 28 x 28 x 6
28 x 28 x 6 ------------------------ Leaky ReLU ------------------------------> 28 x 28 x 6 
28 x 28 x 6 ------- max pooling (kernel size of 2 x 2 with stride = 2) -------> 14 x 14 x 6
14 x 14 x 6 --- conv (16 feature maps of kernel size 5 x 5 and stride = 1) ---> 10 x 10 x 16
10 x 10 x 16 ----------------------- Leaky ReLU ------------------------------> 10 x 10 x 16
10 x 10 x 16 ------ max pooling (kernel size of 2 x 2 with stride = 2) -------> 5 x 5 x 16
5 x 5 x 16  ------------------------- Linear ---------------------------------> 10
```

An example 2 convolutional layers is shown in the image below,

<img src='notebook_ims/2_layer_conv.png' height=50% width=50% />

In [7]:
// Create the Feed Forward Neural Network with Random weight on which our,
// pretrained model weight will be loaded
FFN<NegativeLogLikelihood<>, RandomInitialization> model;

model.Add<Convolution<>>(3, 6, 5, 5, 1, 1, 0, 0, 32, 32); 
model.Add<LeakyReLU<>>(); 
model.Add<MaxPooling<>>(2, 2, 2, 2, true);
model.Add<Convolution<>>(6, 16, 5, 5, 1, 1, 0, 0, 14, 14);
model.Add<LeakyReLU<>>();
model.Add<MaxPooling<>>(2, 2, 2, 2, true);
model.Add<Linear<>>(5*5*16, 120);
model.Add<LeakyReLU<>>();
model.Add<Linear<>>(120, 84);
model.Add<LeakyReLU<>>();
model.Add<Linear<>>(84, 10);
model.Add<LogSoftMax<>>();

###  Load the Model with the Lowest Validation Loss

In [8]:
mlpack::data::Load("model.xml", "model", model);

[WARN ] unrecognized XML syntax


In [9]:
// Matrix for storing test feeature & labels.
mat testData, testY;
// Load the test data.
mlpack::data::Load("../data/cifar-10_test.csv", testData, true);
// Drop the header column.
testData.shed_col(0);
// Remove labels before predicting.
testY = testData.row(testData.n_rows - 1);
testData.shed_row(testData.n_rows - 1);

In [ ]:
cout << "Starting Prediction on testset ..." << endl;
mat testPredProbs;
// Get predictions on test data points.
model.Predict(testData, testPredProbs);
arma::Row<size_t> testPreds = getLabels(testPredProbs);
double testAccuracy = arma::accu(testPreds == testY) / (double) testY.n_elem * 100;

Starting Prediction on testset ...


In [ ]:
cout << "Accuracy on testset = " << testAccuracy << "%" << endl;